<a href="https://colab.research.google.com/github/michel-vbarboza/ERA5-Temperature-Prediction/blob/main/Feedforward_Neural_Network_(GPU).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
#!pip install optuna -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.1 MB/s eta 0:00:00


In [ ]:
import pickle
import pandas as pd
import numpy as np
#import optuna
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error
from tensorflow.keras.mixed_precision import set_global_policy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/ERA5-Temperature-Prediction/Dados/'

# Preprocessing

In [ ]:
with open(path+'train.pkl', 'rb') as f:
    train = pickle.load(f)

with open(path+'test.pkl', 'rb') as f:
    test = pickle.load(f)

del f

In [ ]:
targets = ['target_1', 'target_3', 'target_5',
           'target_7', 'target_10', 'target_15']

features = train.columns.difference(targets)

In [ ]:
target = 'target_10'

In [ ]:
X_train = train[features]
y_train = train[target]

X_test = test[features]
y_test = test[target]

In [ ]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

# Feedforward Neural Network (FNN)

In [ ]:
# Enable GPU memory growth
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# Enable XLA (Accelerated Linear Algebra) compilation
tf.config.optimizer.set_jit(True)

# Set up mixed precision training
set_global_policy('mixed_float16')

# Verify GPU usage and mixed precision
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("Mixed precision policy:", tf.keras.mixed_precision.global_policy())

# Define the neural network model
model = Sequential()
model.add(Input(shape=(X_train.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, dtype='float32'))
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_test, y_test), verbose=1)

# Predict on test data
y_pred = model.predict(X_test)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Root Mean Squared Error on Test Data: {rmse}")


TensorFlow version: 2.17.0
Num GPUs Available:  1
Mixed precision policy: <DTypePolicy "mixed_float16">
Epoch 1/100
13355/13355 ━━━━━━━━━━━━━━━━━━━━ 30s 2ms/step - loss: 4549.1904 - val_loss: 19.9146
Epoch 2/100
13355/13355 ━━━━━━━━━━━━━━━━━━━━ 39s 2ms/step - loss: 14.1419 - val_loss: 24.0933
Epoch 3/100
13355/13355 ━━━━━━━━━━━━━━━━━━━━ 41s 2ms/step - loss: 12.2271 - val_loss: 20.5831
Epoch 4/100
13355/13355 ━━━━━━━━━━━━━━━━━━━━ 29s 2ms/step - loss: 11.3907 - val_loss: 22.3464
Epoch 5/100
13355/13355 ━━━━━━━━━━━━━━━━━━━━ 40s 2ms/step - loss: 10.6648 - val_loss: 29.6471
Epoch 6/100
13355/13355 ━━━━━━━━━━━━━━━━━━━━ 28s 2ms/step - loss: 10.0927 - val_loss: 20.8431
Epoch 7/100
13355/13355 ━━━━━━━━━━━━━━━━━━━━ 26s 2ms/step - loss: 9.7192 - val_loss: 20.1257
Epoch 8/100
13355/13355 ━━━━━━━━━━━━━━━━━━━━ 28s 2ms/step - loss: 9.4642 - val_loss: 25.0013
Epoch 9/100
13355/13355 ━━━━━━━━━━━━━━━━━━━━ 33s 2ms/step - loss: 9.1971 - val_loss: 24.0946
Epoch 10/100
13355/13355 ━━━━━━━━━━━━━━━━━━━━ 27s 2

In [ ]:
# Save predictions and RMSE using pickle
with open(path+'pred_fnn_'+target+'.pkl', 'wb') as f:
    pickle.dump(y_pred, f)

with open(path+'rmse_fnn_'+target+'.pkl', 'wb') as f:
    pickle.dump(rmse, f)